In [1]:
!git clone https://github.com/Rope-player/pytorch_advanced.git

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 548, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 548 (delta 5), reused 162 (delta 0), pack-reused 374
Receiving objects: 100% (548/548), 50.13 MiB | 35.65 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
%cd "pytorch_advanced"

/content/pytorch_advanced


In [3]:
%cd "3_semantic_segmentation"

/content/pytorch_advanced/3_semantic_segmentation


In [4]:
import os
import urllib.request
import zipfile
import tarfile

In [5]:
data_dir = "./data/"
if not os.path.exists(data_dir):
  os.mkdir(data_dir)

In [6]:
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
  os.mkdir(weights_dir)

In [7]:
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar") 

if not os.path.exists(target_path):
  urllib.request.urlretrieve(url, target_path)
    
  tar = tarfile.TarFile(target_path)  
  tar.extractall(data_dir)  
  tar.close()

In [8]:
!gdown https://drive.google.com/uc?id=12eN6SpnawYuQmD1k9VgVW3QSgPR6hICc&export=download   # 학습 된 파라미터 다운로드

Downloading...
From: https://drive.google.com/uc?id=12eN6SpnawYuQmD1k9VgVW3QSgPR6hICc
To: /content/pytorch_advanced/3_semantic_segmentation/pspnet50_ADE20K.pth
100% 197M/197M [00:03<00:00, 60.2MB/s]


In [10]:
!mv /content/pytorch_advanced/3_semantic_segmentation/pspnet50_ADE20K.pth /content/pytorch_advanced/3_semantic_segmentation/weights

In [13]:
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [14]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# 파인튜닝을 활용한 학습 및 검증 실시:

## 학습 및 검증 구현

데이터 로더 작성. 미니 배치 크기는 1gpu 메모리에 담기는 여덟개로 설정.

In [15]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

# 파일 경로 리스트
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)

# 데이터셋 작성
# (RGB)의 평균값과 표준편차
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))

# 데이터로더 작성
batch_size = 8

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 사전형 변수로 함수 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

네트워크 모델을 만들기 위해 먼저 ADE20K 네트워크 모델 준비.

최종 출력층을 Pascal VOC의 21 클래스로 하기 위하여 Decoder와 AuxLoss 모듈의 분류용 합성곱층을 바꿈 

교체한 합성곱 층은 **Xavier의 초기값**으로 초기화.

Xavier의 초기값은 각 합성곱 층에서 입력 채널 수가 `input_n`일때 합성곱층의 결합 파라미터 초기값으로 '`1/sqrt(input_n)`을 표준편차로 한 가우스'에 따라 난수를 사용하는 기법.

이번엔 분류용 유닛의 마지막층이자 활성화 함수인 **시그모이드** 함수 사용. 시그모이드 함수일 경우 Xavier 초기값으로 초기화 함.

In [16]:
from utils.pspnet import PSPNet

# 파인튜닝으로 PSPNet 작성
# ADE20K 데이터셋의 학습된 모델을 사용하여 ADE20 클래스 수는 150
net = PSPNet(n_classes=150)


# ADE20K 학습된 파라미터 읽기
state_dict = torch.load("./weights/pspnet50_ADE20K.pth")
net.load_state_dict(state_dict)


# 분류용 합성곱 층을 출력수 21로 변경
n_classes = 21
net.decode_feature.classification = nn.Conv2d(in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)
net.aux.classification = nn.Conv2d(in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)


# 교체한 합성곱 층을 초기화 . 활성화 함수는 시그모이드 함수이므로 Xavier 사용
def weights_init(m):
  if isinstance(m, nn.Conv2d):
    nn.init.xavier_normal_(m.weight.data)
    if m.bias is not None:  # 바이어스항이 있는 경우
      nn.init.constant_(m.bias, 0.0)


net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)


print('네트워크 설정완료: 학습된 가중치를 로드했습니다.')

네트워크 설정완료: 학습된 가중치를 로드했습니다.


다 클래스 분류의 손실함수 크로스 엔트로피 오차 함수로 손실함수를 구현. 메인 손실과 AuxLoss 합을 총 손실로 합니다. AuxLoss는 계수 0.4를 곱하여 가중치를 메인 손실 보다 작게 함.

In [17]:
class PSPLoss(nn.Module):
    # PSPNet 손실 함수 클래스

  def __init__(self, aux_weight=0.4):
    super(PSPLoss, self).__init__()
    self.aux_weight = aux_weight  # aux_loss 가중치

  def forward(self, outputs, targets):
    # 손실함수 계산

    # Parameters
    #   outputs : PSPNet 출력(tuple) - (output=torch.Size([num_batch, 21, 475, 475]), output_aux=torch.Size([num_batch, 21, 475, 475]))
    #   targets : [num_batch, 475, 475] - 정답 어노테이션 정보
    # Returns
    #   loss    : 텐서 - 손실 값

    loss = F.cross_entropy(outputs[0], targets, reduction='mean')
    loss_aux = F.cross_entropy(outputs[1], targets, reduction='mean')

    return loss+self.aux_weight*loss_aux

criterion = PSPLoss(aux_weight=0.4)

## 스케쥴러로 에폭별 학습 비율 변경

에폭에 따라 학습률을 변화시키는 스케줄러 활용. 코드 `scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)`에 정의 됨.

`lambda_epoch` 함수에 따라 옵티마이저 인스턴스의 학습률을 변화시킴. `lambda_epoch` 함수는 최대 에폭 수를 30으로 하고 에폭을 거칠 때마다 학습률이 서서히 작아지도록 함. `return`하는 값을 옵티마이저 학습률에 곱함.

스케쥴러의 학습률을 변화시키려면 네트워크 학습시 `scheduler.step()`을 실행하면 됨.

In [18]:
optimizer = optim.SGD([
  {'params': net.feature_conv.parameters(), 'lr': 1e-3},
  {'params': net.feature_res_1.parameters(), 'lr': 1e-3},
  {'params': net.feature_res_2.parameters(), 'lr': 1e-3},
  {'params': net.feature_dilated_res_1.parameters(), 'lr': 1e-3},
  {'params': net.feature_dilated_res_2.parameters(), 'lr': 1e-3},
  {'params': net.pyramid_pooling.parameters(), 'lr': 1e-3},
  {'params': net.decode_feature.parameters(), 'lr': 1e-2},
  {'params': net.aux.parameters(), 'lr': 1e-2},
], momentum=0.9, weight_decay=0.0001)


# 스케줄러 설정
def lambda_epoch(epoch):
  max_epoch = 30
  return math.pow((1-epoch/max_epoch), 0.9)


scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)

검증용 함수인 train_model 작성.

2장의 train_model과의 차이점

*   스케줄러가 존재함
*   멀티플 미니 배치 사용

batch_multi

In [19]:
def train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs):

  # GPU를 사용가능한지 확인.
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("使用デバイス：", device)

  # 네트워크를 GPU로
  net.to(device)

  # 네트워크가 어느 정도 고정되면 고속화
  torch.backends.cudnn.benchmark = True

  # 화상 매수
  num_train_imgs = len(dataloaders_dict["train"].dataset)
  num_val_imgs = len(dataloaders_dict["val"].dataset)
  batch_size = dataloaders_dict["train"].batch_size

  # 반복자의 카운터 설정
  iteration = 1
  logs = []

  # 멀티플 미니 배치
  batch_multiplier = 3

  # 에폭 루프
  for epoch in range(num_epochs):

    # 시작 시간 저장
    t_epoch_start = time.time()
    t_iter_start = time.time()
    epoch_train_loss = 0.0  # epoch 손실 합
    epoch_val_loss = 0.0    # epoch 손실 합

    print('-------------')
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-------------')

    # 에폭별 훈련 및 검증 반복문
    for phase in ['train', 'val']:
      if phase == 'train':
        net.train()       # 모델을 훈련 모드로
        scheduler.step()  # 최적화 스케쥴러 갱신
        optimizer.zero_grad()
        print('（train）')

      else:
        if((epoch+1) % 5 == 0):
          net.eval()      # 모델을 검증 모드로
          print('-------------')
          print('（val）')
        else:
          # 검증은 다섯 번 중 한 번만 수행
          continue
      # 데이터로더에서 미니 배치씩 꺼내서 반복
      count = 0  # 멀티플 미니 배치
      for imges, anno_class_imges in dataloaders_dict[phase]:
        # 미니 배치 크기가 1이면 배치 정규화에서 오류가 발생하여 피함.
        if imges.size()[0] == 1:
          continue

        # GPU를 사용할 수 있으면 GPU에 데이터를 보낸다
        imges = imges.to(device)
        anno_class_imges = anno_class_imges.to(device)

        # 멀티플 미니 배치로 파라미터 갱신
        if (phase == 'train') and (count == 0):
          optimizer.step()
          optimizer.zero_grad()
          count = batch_multiplier

        # 순전파 계산
        with torch.set_grad_enabled(phase == 'train'):
          outputs = net(imges)
          loss = criterion(outputs, anno_class_imges.long()) / batch_multiplier

          # 훈련시에는 역전파
          if phase == 'train':
            loss.backward()  # 경사 계산
            count -= 1       # 멀티플 미니 배치

            if (iteration % 10 == 0):  # 10iter에 한 번 손실 표시
              t_iter_finish = time.time()
              duration = t_iter_finish - t_iter_start
              print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(iteration, loss.item()/batch_size*batch_multiplier, duration))
              t_iter_start = time.time()

            epoch_train_loss += loss.item() * batch_multiplier
            iteration += 1

          # 검증
          else:
            epoch_val_loss += loss.item() * batch_multiplier
    # 에폭의 페이즈별 손실과 정답률
    t_epoch_finish = time.time()
    print('-------------')
    print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
    print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
    t_epoch_start = time.time()

    # 로그 저장
    log_epoch = {'epoch': epoch+1, 'train_loss': epoch_train_loss / num_train_imgs, 'val_loss': epoch_val_loss/num_val_imgs}
    logs.append(log_epoch)
    df = pd.DataFrame(logs)
    df.to_csv("log_output.csv")


  # 마지막 네트워크 저장
  torch.save(net.state_dict(), 'weights/pspnet50_' + str(epoch+1) + '.pth')

In [ ]:
# 학습 및 검증 시행
num_epochs = 30
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs=num_epochs)